# 过拟合和欠拟合

和通常情况一样，在这个例子中代码将使用 `tf.keras` API.

在前面的*基本回归*例子中，我们看到模型在验证数据上的精度在经过多轮训练后将停止或开始降低。.

换句话说，我们的模型将*overfit过拟合* 训练数据. 学习如何处理过拟合是重要的。尽管在训练集上可以获得很高的精确度，但是我们开发模型真正需要的是在测试集（或我们没有看到的数据）上泛化良好。

过拟合的对立面是*underfitting欠拟合*. 当在测试数据上还有改进空间时，欠拟合发生。有多个原因可能导致发生欠拟合：模型不够强大，过度正则化，没有足够训练等。这意味着网络没有学到训练数据中相关模式。

如果你训练太长，模型将开始过拟合，并从训练数据中学到不能泛化到测试数据上的模式。我们需要获得平衡。如同我们下面介绍的，理解如何训练适当的轮数是一个有用的技巧。

为了防止过拟合，最好的方案是使用更完整的数据。数据集应该覆盖模型期望处理的输入的范围。如果模型覆盖新的有趣清晰，额外数据也是有帮助的。

一个在完整数据上训练的模型将自然的泛化良好。当这不可能时，次佳方案是使用像正则化这样的技术。这将对模型可以存储的信息的数量和类型进行限制，优化过程将强制模型关注重要的模式，这些模式更有机会泛化良好。

本节将展示几种常见的正则化技术，并使用它们改进分类模型。

## 设置

在开始前，导入必要的包：

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

In [ ]:
# 注释掉。无法安装。从github中下载tensorflow-docs将，tensorflow-docs目录拷到site-packages
# !pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [ ]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile


In [ ]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)


## Higgs数据集

我们的目标不是介绍量子物理，因此不需要太在意数据集的细节，它包含11000000个样本，每个有28个特征和一个二元类标签。

In [ ]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 
                             'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz')

In [ ]:
FEATURES = 28

类`tf.data.experimental.CsvDataset`可以用来不需要中间解压就直接从gzip文件中读取csv记录。 

In [ ]:
ds = tf.data.experimental.CsvDataset(gz,
                                     [float(),]*(FEATURES+1), 
                                     compression_type="GZIP")

csv读取器返回每个记录的标量列表，下列函数将标量列表转换成（特征向量，标签）对。

In [ ]:
def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label

TensorFlow在大批量数据上操作效率最高。 

因此，不是在每行记录上转换创建一个新数据集，而是对10000条记录的数据批进行转换，然后将数据批再转换成单个记录。

In [ ]:
packed_ds = ds.batch(10000).map(pack_row).unbatch()

可以看看`packed_ds`中数据。

特征没有规范化，但这对教材足够了。

In [ ]:
for features,label in packed_ds.batch(1000).take(1):
  print(features[0])
  plt.hist(features.numpy().flatten(), bins = 101)

为了使训练相对较短，我们只使用前1000条样本作为验证集，接下来的10000条样本作为训练集。

In [ ]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

方法 `Dataset.skip` 和 `Dataset.take` 使这很容易实现。

同时，使用方法 `Dataset.cache` 确保装载器不必在每轮从文件中重新装载数据。

In [ ]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

In [ ]:
train_ds

这些数据集返回单个样本，使用方法 `.batch`创建适当大小的训练批数据，在成批前记得 `.shuffle` 和 `.repeat`训练集。

In [ ]:
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

## 过拟合展示

防止过拟合的最简单方法是先使用小模型：一个有比较少学习参数的模型（由层数和每层单元数决定）。在深度学习中，模型的可学习参数通常称为模型的容量。

直觉上，一个有更多参数的模型将有更多的记忆容量，因而能够容易学习一个完美的词典--像从训练样本到目标的映射，不需要任何泛化能力的映射，当在没有看到的数据上预测时，这将没有用处。

经常记住：深度学习模型趋向于很好的拟合训练数据，但是真正挑战是泛化，而不是拟合。
另外一方面，如果网络记忆资源有限，它将不能容易的学习到映射。为了最小化损失，模型将不得不学习有更强预测能力的压缩表示。同时，如果你将模型设置太小，它将难以拟合训练数据。在容量太大和容量不足间存在一个平衡。

不幸的是，没有魔法公式确定模型的正确架构和大小，即层数和每层大小。你必须用不同的架构进行实验。

为了发现一个合适的模型大小，最好先从相对较少的层和参数开始，然后增加层的大小或添加新的层，直到你发现验证损失变大。

先使用只包含`layers.Dense`的模型作为基准，然后创建大模型，比较它们。

### 训练过程

如果在训练过程中逐步降低学习率，许多模型训练会更好。使用 `optimizers.schedules`逐步减低学习率。 

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

以上代码设置了 `schedules.InverseTimeDecay`，它以双曲线的方式在训练过程中降低学习率。

In [ ]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')


本教程中每个模型将使用相同的训练配置。因此，以可复用的方式设置一个回调callback列表.

本教程的训练轮数较短。为了减少日志信息，使用`tfdocs.EpochDots`，它每轮只输出一个 `.`，每100轮输出完整的指标值。

使用`callbacks.EarlyStopping`来避免不必要的训练时间，这个回调监视`val_binary_crossentropy`, 而不是 `val_loss`. 这个不同在后面将很重要.

使用`callbacks.TensorBoard`来产生TensorBoard训练日志。

In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

类似的，每个模型将使用相同的 `Model.compile` 和 `Model.fit` 设置:

In [ ]:
def compile_and_fit(model, name, optimizer=None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()
  model.compile(optimizer=optimizer,
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[
                  tf.keras.losses.BinaryCrossentropy(
                      from_logits=True, name='binary_crossentropy'),
                  'accuracy'])

  model.summary()

  history = model.fit(
    train_ds,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_data=validate_ds,
    callbacks=get_callbacks(name),
    verbose=0)
  return history

### 微模型

训练一个小模型:

In [ ]:
tiny_model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(1)
])

In [ ]:
size_histories = {}

In [ ]:
size_histories['Tiny'] = compile_and_fit(tiny_model, 'sizes/Tiny')

接下来检测模型表现如何:

In [ ]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)
plt.ylim([0.5, 0.7])

### 小模型

现在训练一个大一点的模型，看它能否比前面的小模型的性能好。

试试带16个单元的两个隐藏层的模型:

In [ ]:
small_model = tf.keras.Sequential([
    # `input_shape` is only required here so that `.summary` works.
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(16, activation='elu'),
    layers.Dense(1)
])

In [ ]:
size_histories['Small'] = compile_and_fit(small_model, 'sizes/Small')

### 中等模型

现在试试带64个单元的3个隐藏层的模型：

In [ ]:
medium_model = tf.keras.Sequential([
    layers.Dense(64, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(1)
])

使用相同的数据训练模型:

In [ ]:
size_histories['Medium']  = compile_and_fit(medium_model, "sizes/Medium")

### 大模型

作为练习，你可以创建更大的模型，并看到它如何快速的开始过拟合。接下来，我们创建一个有更大容量的网路，它比我们解决问题需要的网络容量大得多:

In [ ]:
large_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(1)
])

同样，使用相同的数据训练模型:

In [ ]:
size_histories['large'] = compile_and_fit(large_model, "sizes/large")

### 绘制训练和验证损失

实线表示训练损失，点线表示验证损失（记住：小的验证损失表示好的模型）.

较大的模型有更大的能力，但如果不约束它，较大模型可能更容易过拟合训练数据.

在这个例子中，只有微模型设法避免过拟合，每个大一些的模型更快的过拟合数据。这对于大模型更加严重，你需要将图的坐标变成对数坐标才能看到发生什么。

如果你绘图比较验证指标和训练指标时，很明显：

* 通常这里会有一些不同。
* 如果所有指标向相同方向移动，这很好.
* 如果验证指标开始停止不前，而训练指标持续改善，你可能离过拟合比较近了。
* 如果验证指标开始变坏，你的模型一定是过拟合了。

In [ ]:
plotter.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])
plt.xlabel("Epochs [Log Scale]")

注意：所有上面训练都使用了`callbacks.EarlyStopping`来结束训练， 当模型不再改善时。

### 在TensorBoard中查看

这些模型在训练过程中写了TensorBoard日志

在notebook中打开嵌入式TensorBoard查看器。

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir {logdir}/sizes

你可以在[TensorBoard.dev](https://tensorboard.dev/)上查看该笔记本以前的运行结果[以前运行结果](https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97) .

TensorBoard.dev是一个保存、跟踪和共享机器学习实验经验的地方。 

为了方便，它也包含在一个 `<iframe>`中:

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97",
    width="100%", height="800px")

## 防止过拟合的策略

在开始这部分内容前，拷贝前面的微模型的训练日志，并将微模型作为比较基准。

In [ ]:
shutil.rmtree(logdir/'regularizers/Tiny', ignore_errors=True)
shutil.copytree(logdir/'sizes/Tiny', logdir/'regularizers/Tiny')

In [ ]:
regularizer_histories = {}
regularizer_histories['Tiny'] = size_histories['Tiny']

### 添加权重规范化或正则化

你可能熟悉Occam剃刀法则：对于某件事的两个解释，最可能正确的解释往往是最简单的，它有最小的假设。这同样适合于神经网络学习到的模型：给定相同训练数据和网络结构，有很多不同的权重或模型可以解释数据，简单模型相比复杂模型更不易过拟合数据。

在这里，简单模型的参数值分布有较少的熵（或有较少参数）。因此，一个降低过拟合的简单方法是，对网络复杂性设定限制，强迫它的权重取较少的值，这使得权重值的分布比较规范或正则。这称为权重规范化或正则化，它通过向网络的损失函数中增加一个和权重有关的代价实现，较大的权重有较大的代价。这个代价有两种方法计算：

* [L1正则化], 添加的代价正比于权重的绝对值 (即，权重的L1模).

* [L2正则化], 添加的代价正比于权重的平方（即，权重的L2模）. L2正则化在神经网络中也称为权重衰减。不要让这些名字迷惑你：权重衰减在数学上和L2正则化一样。

L1正则化将权重推向0，产生一个稀疏的模型。L2正则化同样惩罚大的权重，但没有要求模型稀疏，因为小的权重的惩罚趋近于0。 

在`tf.keras`中, 权重正则化通过将权重正则化实例作为关键字参数添加到层中。我们现在添加L2权重正则化.

In [ ]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1)
])

regularizer_histories['l2'] = compile_and_fit(l2_model, "regularizers/l2")

`l2(0.001)`表示在权重矩阵中的每个权重将添加`0.001 * weight_coefficient_value**2`到网络的**损失**中。

这是我们为什么监视 `binary_crossentropy`的原因，因为，它没有正则化混在其中。

因此，带L2正则化的大模型表现好些：

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

正如你所看到的，L2正则化模型和微模型相比有竞争力。L2正则化模型也相对于大模型来说更难出现过拟合，即使它们有相同数量的参数。

#### 更多信息

关于正则化有两个重要事情需要注意。

**第一:** 如果你自己写训练过程，你需要确保模型加入正则损失。

In [ ]:
result = l2_model(features)
regularization_loss=tf.add_n(l2_model.losses)

**第二:** 向模型损失中加入权重惩罚，然后使用标准的优化过程。

有第二种方法，只在原始损失上优化，然后优化器同时应用某些权值衰减。这种*解耦权值衰减*在优化器 `optimizers.FTRL` 和 `optimizers.AdamW`中可以看到。

### 添加dropout

Dropout是一种神经网络最有效和最常见的正则化技术，由Hinton和他的学生在多伦多大学开发。

Dropuout的直觉解释是，因为网络中单个节点不能依赖其他节点的输出，所以每个节点必须输出它们有用的特征。

应用于层的Dropout在训练时随机对层的输出特征丢弃或置0. 假设某层在训练时对给定输入样本正常返回向量 [0.2, 0.5, 1.3, 0.8, 1.1]，在应用Dropout后，向量的某些分量将随机置0，结果可能是 [0, 0.5,
1.3, 0, 1.1].

丢弃率是丢弃或置0的特征比例，它通常在0.2到0.5间。在测试的时候，没有单元丢弃，层的输出值根据丢弃率进行扩展，以便相比于训练时有更多单元激活。

在`tf.keras`中，你可以通过Dropout层来引入dropout，它对前一层的输出应用dropout。

我们添加两个Dropout层到网络中，并看看它们如何降低过拟合：

In [ ]:
dropout_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['dropout'] = compile_and_fit(dropout_model, "regularizers/dropout")

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

从图形中可以看出，这些正则化方法改善了大模型的性能，但是甚至不能好过微模型基准。

现在组合它们，我们将看到更好的结果.

### 组合L2 规范化和 dropout

In [ ]:
combined_model = tf.keras.Sequential([
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1)
])

regularizer_histories['combined'] = compile_and_fit(combined_model, "regularizers/combined")

In [ ]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

这个组合正则化的模型显然是目前最好的模型。

### 在TensorBoard中查看

这些模型记录了TensorBoard日志.

为了在notebook中打开嵌入式tensorboard，拷贝下列内容到一个代码格中:

```
%tensorboard --logdir {logdir}/regularizers
```

你可以在[TensorDoard.dev](https://tensorboard.dev/)中浏览本notebook[以前运行结果](https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97) .

为了方便，它们也包括在 `<iframe>` 中:

In [ ]:
display.IFrame(
    src="https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97",
    width = "100%",
    height="800px")


下列代码将上传日志:

```
!tensorboard dev upload --logdir  {logdir}/regularizers
```

## 总结

回顾一下: 以下是在神经网络中预防过拟合的常见方法:

* 获得更多训练数据.
* 降低网络容量.
* 增加权重规范化或正则化.
* 添加dropout.

本部分没有涉及的两个重要方法是:

* 数据增强data-augmentation
* 批规范化batch normalization

记住每个方法有它们的作用，但是组合它们常可能更加有用.